<a href="https://colab.research.google.com/github/uteyechea/neural-network-from-scratch/blob/main/nn_model_2layers-fake_softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Part 1

##1.1 Import all necessary dependencies

In [1]:
!pip install scikit-multilearn #Data split train/test sets

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix
from skmultilearn.model_selection import iterative_train_test_split

##1.2 Prepare the dataset

In [3]:
#Binary classifier toy data, could be XOR, AND, OR, ...

# Three training examples with two features x1 and x2.
X=np.array([
    [0,0,1,1],
    [0,1,0,1]
])

print('# Training examples, m: {}'.format(X.shape[1]))
print('# of features, n_x: {}'.format(X.shape[0]))
#Labels for each of the three training examples.
Y=np.array([
            [1,0,0,1],
])

# Visualize the data
fig=go.Figure(data=go.Scatter(x=X[0,:],
                                y=X[1,:],
                                mode='markers',
                                marker_color=Y[0,:],
                                marker_size=25,
                                text=Y[0,:]))
fig.update_layout(title='Binary classification training samples')
fig.show()

# Training examples, m: 4
# of features, n_x: 2


## 1.3 ECG data
1.   Download data
2.   Split data into train and test sets
3.   Verify balanced distribution of data labels



### 1.3.1 Download data

In [4]:
# Download ECG data
data_url='https://github.com/uteyechea/neural-network-from-scratch/blob/main/ecg_data/ecg.csv'

def read_cvs_with_html_tags(data_url): 
  # This function only works for my very peculiar data structure.
  """
  Arguments:
  data_url -- csv file url

  Returns:
  X -- Training data
  y -- Labels
  """ 
  raw_data=pd.read_html(data_url) # Looking for <table> tag
  unformatted_data=raw_data[0][1].str.split(";",expand=True) #read_html return need a little work
  #By now you have a standard pandas DataFrame, but still needs some work...
  #make first row new header, (optional, depending on your csv file)
  new_header=unformatted_data.iloc[0,:]
  data=unformatted_data[1:]
  data.columns=new_header

  X=data.iloc[:,0:-5]
  y=data.loc[:,'Clase']
  y=y.values.reshape(y.shape[0],1)

  X=np.array(X)
  y=np.array(y)

  X=X.astype(np.float)
  y=y.astype(np.float)

  y=y/max(y)-(1/max(y))

  return X,y

###1.3.2 Split train/test sets

In [5]:
#data_url='https://github.com/uteyechea/neural-network-from-scratch/blob/main/ecg_data/ecg.csv'
#X,y=read_cvs_with_html_tags(data_url)

In [6]:
#X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size = 0.5)

###1.3.3 Verify balanced distribution of labels

In [7]:
def count_train_test_split(y_test,y_train):
  #Assumed y_train, y_test data shape (n,1)

  y_test_distribution={}
  y_train_distribution={}

  for label in np.unique(y_test):
    y_test_distribution['class='+str(label)]= np.count_nonzero(y_test == label, axis=0)
  
  for label in np.unique(y_train):
    y_train_distribution['class='+str(label)]= np.count_nonzero(y_train == label, axis=0)
  
  return y_train_distribution,y_test_distribution

###1.3.4 Get data

In [8]:
def get_ecg_data():
  """
  Agruments(None):
  data_url -- csv file location
  test_size -- Test/train data split, ex. test_size=0.7 distributes 70% to the test set and 30% to the training set. 
  Returns:
  X_train -- train data features
  y_train -- train data labels
  X_test -- test data features
  y_test -- test data labels
  """
  data_url='https://github.com/uteyechea/neural-network-from-scratch/blob/main/ecg_data/ecg.csv'
  X,y=read_cvs_with_html_tags(data_url)

  print(y.shape)
  X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size = 0.5) 
  y_train_distribution,y_test_distribution=count_train_test_split(y_test,y_train)
  print(y_train_distribution)
  print(y_test_distribution)

  #Reshape data to fit expected data structure for nn model
  X_train=X_train.reshape(X_train.shape[1],X_train.shape[0])
  X_test=X_test.reshape(X_test.shape[1],X_test.shape[0])
  y_train=y_train.reshape(y_train.shape[1],y_train.shape[0])
  y_test=y_test.reshape(y_test.shape[1],y_test.shape[0])

  N=len(np.unique(y_test))
  print('distinct labels: ',N )

  return X_train, y_train, X_test, y_test,N

In [9]:
X_train, y_train, X_test, y_test,N = get_ecg_data()

(460, 1)
{'class=0.0': array([96]), 'class=0.25': array([46]), 'class=0.5': array([46]), 'class=0.75': array([42])}
{'class=0.0': array([96]), 'class=0.25': array([45]), 'class=0.5': array([46]), 'class=0.75': array([43])}
distinct labels:  4


In [10]:
print(y_test.shape)
print(X_train.shape)
print('# Training examples, m: {}'.format(X_train.shape[1]))
print('# of features, n_x: {}'.format(X_train.shape[0]))


print(np.sum(X_train),
np.sum(X_test),
np.sum(y_train),
np.sum(y_test)
)


(1, 230)
(39, 230)
# Training examples, m: 230
# of features, n_x: 39
3826.0886028808795 3638.479131882022 66.0 66.5


#Part 2

#2.1 Define layer sizes

In [11]:
def layer_sizes(X, Y):
    """
    Arguments:
    X -- input dataset of shape (number of features, number of training examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer, i. e., number of features
    n_h -- the size of the hidden layer, i. e., number of neurons
    n_y -- the size of the output layer, i. e., label
    """
    
    n_x = X.shape[0] # size of input layer, i. e. number of features
    n_h = 3 # user defined number of neurons inside the one and only layer
    n_y = Y.shape[0] # size of output layer, i. e. number of possible labels
    
    return (n_x, n_h, n_y)

#2.2 Initialize model parameters

In [12]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer, i. e., number of features
    n_h -- size of the hidden layer, i. e., number of neurons
    n_y -- size of the output layer, i. e., label
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h) #if Second layer is the output ot the NN.
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    W1 = np.random.randn(n_h,n_x) * np.sqrt(1./ (n_x))#0.01 it is usually recommended to start with small weights, as long as they are different than zero
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h) * np.sqrt(1./ (n_h))
    b2 = np.zeros((n_y,1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

#2.3 Implement forward propagation

In [13]:
def forward_propagation(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m) #where m is the number of training examples
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    # Retrieve each parameter from the dictionary "parameters"
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    # Implement Forward Propagation to calculate A2 (probabilities)
    Z1 = W1 @ X + b1 #(n_h,n_x) @ (n_x,m) + (n_h,1) = (n_h,m)
    A1 = np.tanh(Z1) #(n_h,m) selected activation function for the hidden layer neurons
    Z2 = W2 @ A1 + b2 
    A2 = 1.0 / (1.0 + np.exp(-Z2)) #NN output, later we will round this value to 0/1, thus the sigmoid function fits the expected output map just fine
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

### 2.2.1 Compute Cost

In [14]:
def compute_cost(A2, Y):
    """
    Computes the cross-entropy cost. It is prefered to use this cost function given it is stricly convex 
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (output size, number of examples)
    Y -- "true" labels vector of shape (output size, number of examples)
    
    Returns:
    cost -- cross-entropy cost
    
    """
    
    m = Y.shape[1] # get the number of training examples

    # Compute the cross-entropy cost
    logprobs = Y*np.log(A2) + (1.0-Y) * np.log(1.0-A2) # (n_y,m) * (n_h,m) + ... = (n_h,m)
    cost = -(1/m)*np.sum(logprobs,axis=1,keepdims=True) # sum the cost over all training examples
    
    #cost = float(np.squeeze(cost))  # makes sure cost is the dimension we expect                               
    #assert(isinstance(cost, float))

    return cost

#Part 3

#3.1 Implement Backpropagation

In [15]:
def backward_propagation(parameters, cache, X, Y):
    """
    Implement the backward propagation
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1] #get number of training examples
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    W1 = parameters['W1']
    W2 = parameters['W2']
        
    # Retrieve also A1 and A2 from dictionary "cache".
    A1 = cache['A1']
    A2 = cache['A2']
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    dZ2 = A2-Y
    dW2 = (1/m)*dZ2@A1.T #(n_h,m) @ (n_h,m) = (n_h,n_h)
    db2 = (1/m)*np.sum(dZ2,axis=1,keepdims=True)
    dZ1 = W2.T@dZ2 * (1 - np.power(A1, 2)) 
    # (n_h,n_x).T @ (n_h,m) * (n_h,m) - (n_y,m) = (n_x,m) * (n_h,m) Broadcasting
    dW1 = (1/m)*dZ1@X.T #(n_h,m) * (n_h,m) = (n_h,m)
    db1 = (1/m)*np.sum(dZ1,axis=1,keepdims=True)
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

#Part 4

#4.1 Update parameters

In [16]:
def update_parameters(parameters, grads, learning_rate):
    """
    Updates parameters using the gradient descent update rule. Ex: w= w-alpha*dw, b= b-alpha*db
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    # Retrieve each parameter from the dictionary "parameters"
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    # Retrieve each gradient from the dictionary "grads"
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    
    # Update rule for each parameter
    W1 = W1-learning_rate*dW1
    b1 = b1-learning_rate*db1
    W2 = W2-learning_rate*dW2
    b2 = b2-learning_rate*db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

#Part 5

##5.1 Build NN model - 2 layers

In [17]:
def nn_model(X, Y, n_h, num_iterations = 10000, learning_rate=0.01, print_cost=False):
    """
    Arguments:
    X -- dataset of shape (2, number of examples)
    Y -- labels of shape (1, number of examples)
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    learning_rate -- step size while implementing gradient descent
    print_cost -- if True, print the cost every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    # Initialize parameters
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):
         
        # Forward propagation. Inputs: "X, parameters". Outputs: "A, cache".
        A, cache = forward_propagation(X, parameters)
        
        #print(A)
        # Cost function. Inputs: "A, Y, parameters". Outputs: "cost".
        cost = compute_cost(A, Y)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward_propagation(parameters, cache, X, Y)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            #print ("Cost after iteration %i: %f" %(i, cost))
            print('cost',i,cost)

    return parameters

#5.2 Make function to generate predictions

In [18]:
def predict(parameters, X, N):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m), could be test set, validation set or new observations
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    A, cache = forward_propagation(X, parameters)
    #predictions = np.around(A,0) # integer class labels 0/1
    
    print(A.shape)

    Y=A.reshape(A.shape[1],A.shape[0])

    predictions=[]
    for element in Y:
      for n in range(0,int(N)+1):
        if n/N < element <= (n+1)/N:
          prediction=(n)/N#np.around( n+1/N, 2)
          predictions.append(prediction)
          #print(scaled_y) 
          #break 

    predictions=np.array(predictions)
    predictions=predictions.reshape(A.shape[0],A.shape[1])

    print(predictions.shape)
    
    return predictions

In [19]:

for n in range(0,int(N)+1):
  print(n)

0
1
2
3
4


# Part 6: Run model

In [20]:
# Build a model with a n_h-dimensional hidden layer
parameters = nn_model(X_train, y_train, n_h = 128, num_iterations = 30000, learning_rate=0.1, print_cost=True)

cost 0 [[0.78066212]]
cost 1000 [[0.50080836]]
cost 2000 [[0.42755274]]
cost 3000 [[0.39591756]]
cost 4000 [[0.38227131]]
cost 5000 [[0.37491526]]
cost 6000 [[0.37028789]]
cost 7000 [[0.36708907]]
cost 8000 [[0.36474733]]
cost 9000 [[0.36297744]]
cost 10000 [[0.36160606]]
cost 11000 [[0.36052234]]
cost 12000 [[0.35965221]]
cost 13000 [[0.35894812]]
cost 14000 [[0.35838653]]
cost 15000 [[0.35818914]]
cost 16000 [[0.35841346]]
cost 17000 [[0.35863634]]
cost 18000 [[0.35859229]]
cost 19000 [[0.35841153]]
cost 20000 [[0.35819529]]
cost 21000 [[0.35798002]]
cost 22000 [[0.35777613]]
cost 23000 [[0.35758618]]
cost 24000 [[0.35741024]]
cost 25000 [[0.35724753]]
cost 26000 [[0.357097]]
cost 27000 [[0.35695757]]
cost 28000 [[0.35682818]]
cost 29000 [[0.35670787]]


In [21]:
# Print accuracy
predictions = predict(parameters, X_train, N)
print ('Accuracy: %d' % float((np.dot(y_train,predictions.T) + np.dot(1-y_train,1-predictions.T))/float(y_train.size)*100) + '%')


(1, 230)
(1, 230)
Accuracy: 75%


In [22]:
print(predictions.shape,
y_train.shape)
y_train-predictions

(1, 230) (1, 230)


array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 

In [23]:
# Confusion matrix
#sklearn.metrics.confusion_matrix(y_true, y_pred, *, labels=None, sample_weight=None, normalize=None)
confusion_matrix(y_train.T*N,predictions.T*N)

array([[96,  0,  0,  0],
       [ 2, 44,  0,  0],
       [ 0,  0, 46,  0],
       [ 0,  0,  0, 42]])

#Part 7: Verify over/under fitting by the model 

In [24]:
def nn_model(X_train, y_train, X_test, y_test, n_h, num_iterations = 10000, learning_rate=0.1, print_cost=False):
    """
    Arguments:
    X -- dataset of shape (2, number of examples)
    Y -- labels of shape (1, number of examples)
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    learning_rate -- step size while implementing gradient descent
    print_cost -- if True, print the cost every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    n_x = layer_sizes(X_train, y_train)[0]
    n_y = layer_sizes(X_train, y_train)[2]
    
    # Initialize parameters
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):


        # Forward propagation. Inputs: "X, parameters". Outputs: "A, cache".
        A, cache = forward_propagation(X_train, parameters)
        # Cost function. Inputs: "A, Y, parameters". Outputs: "cost".
        cost = compute_cost(A, y_test)

        # Forward propagation. Inputs: "X_test, parameters". Outputs: "A_validation, cache_validation".
        A_validation,cache_validation=forward_propagation(X_test, parameters)
        # Cost function. Inputs: "A_validation, Y_test, parameters". Outputs: "cost_validation".
        cost_validation = compute_cost(A_validation, y_test)


        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward_propagation(parameters, cache, X_train, y_train)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost),'Validation cost %i:%f'%(i,cost_validation) )

    return parameters

In [25]:
parameters = nn_model(X_train, y_train,X_test,y_test, n_h = 128, num_iterations = 30000, learning_rate=0.1, print_cost=True)

Cost after iteration 0: 0.743041 Validation cost 0:0.743780
Cost after iteration 1000: 0.511184 Validation cost 1000:0.588599
Cost after iteration 2000: 0.432031 Validation cost 2000:0.564453
Cost after iteration 3000: 0.398656 Validation cost 3000:0.570238
Cost after iteration 4000: 0.384467 Validation cost 4000:0.580373
Cost after iteration 5000: 0.376560 Validation cost 5000:0.591831
Cost after iteration 6000: 0.371640 Validation cost 6000:0.603662
Cost after iteration 7000: 0.368301 Validation cost 7000:0.615351
Cost after iteration 8000: 0.365942 Validation cost 8000:0.626844
Cost after iteration 9000: 0.364101 Validation cost 9000:0.638007
Cost after iteration 10000: 0.362736 Validation cost 10000:0.648112
Cost after iteration 11000: 0.361663 Validation cost 11000:0.657638
Cost after iteration 12000: 0.360829 Validation cost 12000:0.666452
Cost after iteration 13000: 0.360210 Validation cost 13000:0.674638
Cost after iteration 14000: 0.359744 Validation cost 14000:0.682577
Cost a

In [26]:
# Print accuracy
predictions = predict(parameters, X_test, N)
print ('Accuracy: %d' % float((np.dot(y_test,predictions.T) + np.dot(1-y_test,1-predictions.T))/float(y_test.size)*100) + '%')


(1, 230)
(1, 230)
Accuracy: 69%


In [27]:
print(predictions.shape,
y_test.shape)

y_test-predictions

(1, 230) (1, 230)


array([[-0.25,  0.  ,  0.  ,  0.  , -0.75,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.75,  0.  ,
         0.  , -0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        -0.75,  0.  , -0.5 ,  0.  ,  0.  , -0.75,  0.  , -0.25,  0.  ,
        -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.25,  0.  ,
        -0.25, -0.5 , -0.25, -0.5 , -0.75,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  , -0.25, -0.25, -0.25,  0.  ,  0.  ,  0.  ,
         0.  , -0.5 ,  0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.25,  0.25,  0.25,
         0.25,  0.25, -0.5 ,  0.  , -0.25,  0.  ,  0.  , -0.25,  0.25,
         0.25,  0.  ,  0.25,  0.25,  0.25,  0.25, -0.5 ,  0.  ,  0.  ,
         0.25,  0.25,  0.25,  0.25,  0.  ,  0.25,  0.  ,  0.25,  0.25,
      

In [28]:
# Confusion matrix
#sklearn.metrics.confusion_matrix(y_true, y_pred, *, labels=None, sample_weight=None, normalize=None)
confusion_matrix(y_test.T*N,predictions.T*N)

array([[76, 10,  4,  6],
       [30, 10,  3,  2],
       [15,  6, 13, 12],
       [15,  9, 10,  9]])